In [23]:
import import_ipynb
import cv2
import matplotlib.pyplot as plt
import numpy as np
from search import *

# 볼링 점수판에서 총 점수만 잘라내는 함수
def cut_score(image):
    link = image
    image = cv2.imread(image)
    H, W, C = image.shape

    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_blurred = cv2.GaussianBlur(img_gray, ksize = (5,5), sigmaX = 0)
    img_thresh = cv2.adaptiveThreshold(img_blurred, maxValue = 255.0,
                                      adaptiveMethod = cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      thresholdType = cv2.THRESH_BINARY_INV,
                                      blockSize = 19, C = 9)

    contours, _ = cv2.findContours(img_thresh, mode = cv2.RETR_LIST, method = cv2.CHAIN_APPROX_SIMPLE)

    temp_result = np.zeros((H, W, C), dtype = np.uint8)

    cv2.drawContours(temp_result, contours = contours, contourIdx = -1, color = (255,255,255))
#     plt.figure(figsize= (12,10))
#     plt.imshow(temp_result)

    contours_dict = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(temp_result, pt1 = (x,y), pt2 = (x+w, y+h),
                     color = (255,255,255), thickness = 3)
        contours_dict.append({
            'contour' : contour,
            'x' : x,
            'y' : y,
            'w' : w,
            'h' : h,
            'cx' : x + (w/2),
            'cy' : y + (h/2)
        })

#     plt.figure(figsize=(12,10))
#     plt.imshow(temp_result, cmap = 'gray')

    MIN_AREA = 15000
    MAX_AREA = 300000
    MIN_WIDTH, MIN_HEIGHT = 300, 100
    MIN_RATIO, MAX_RATIO = 0.1, 5

    possible_contours = []

    cnt = 0

    for d in contours_dict:
        area = d['w'] * d['h']
        ratio = d['w'] / d['h']

        if area < MAX_AREA and area > MIN_AREA and d['w'] > MIN_WIDTH and d['h'] > MIN_HEIGHT and MIN_RATIO < ratio < MAX_RATIO:
            d['idx'] = cnt
            cnt += 1
            possible_contours.append(d)

    temp_result = np.zeros((H, W, C), dtype = np.uint8)

    for d in possible_contours:
        cv2.rectangle(temp_result, pt1 = (d['x'], d['y']), pt2 = (d['x']+d['w'], d['y']+d['h']),
                     color = (255,255,255), thickness = 2)
    
#     plt.figure(figsize = (12,10))
#     plt.imshow(temp_result, cmap = 'gray')
    
    temp = []
    
    # 중복 제거하기 !!!!!!!!!!!!!!!!!!!!
    for i in range(len(possible_contours)):
        d1 = possible_contours[i]
        for j in range(i+1, len(possible_contours)):
            d2 = possible_contours[j]
            if d2['cy'] -10 < d1['cy'] and d1['cy'] < d2['cy'] + 10:
                temp.append(d1)
                break
    
    plate_img = []
    
    # len(temp) == 0 인 경우 -> 스코어가 1개일 경우 혹은 중복인 경우가 없는 경우.
    if len(temp) == 0:
        for d in possible_contours:
            pw = d['w']
            ph = d['h']
            pcx = d['cx']
            pcy = d['cy']

            img_cropped = cv2.getRectSubPix(
                image,
                patchSize = (pw, ph),
                center = (pcx, pcy)
            )

            plate_img.insert(0, img_cropped)
    else:
        for d in temp:
            pw = d['w']
            ph = d['h']
            pcx = d['cx']
            pcy = d['cy']

            img_cropped = cv2.getRectSubPix(
                image,
                patchSize = (pw, ph),
                center = (pcx, pcy)
            )

            plate_img.insert(0, img_cropped)

    
#     for i in range(len(plate_img)):
#         plt.subplot(1, len(plate_img), i+1)
#         plt.imshow(plate_img[i], cmap='gray')
#         plt.axis('off')
        
    return plate_img


In [24]:
# images = search('./board')
# cnt = 0
# save_path = './score/'
# for img in images:
#     plate_img = cut_score(img)
#     index = 0
#     for data in plate_img:
#         data = cv2.resize(data, (200, 100))
#         cv2.imwrite(save_path + str(cnt) + '_' + str(index) + '.jpg', data)
#         index += 1
#     cnt += 1